In [1]:
%load_ext autoreload
%autoreload 2
!source /home/murilo/RelNetCare/.env

In [2]:
# import hunspell
import re
import torch
import language_tool_python
from transformers import pipeline

def extract_bot_reply(outputs, bot_name='Adele'):
    # Extracts raw response from full generated text
    raw_response = outputs[0]["generated_text"].split('<|assistant|>\n')[-1]

    # Define the pattern to extract the bot's reply
    pattern = rf'{bot_name}: ([^\(\n]*)'

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, raw_response)

    # Return the first match stripped of leading/trailing whitespace, or None if no match
    return matches[0].strip() if matches else None


def correct_text(input_text, language='de-DE'):
    # Initialize the LanguageTool object with the specified language
    tool = language_tool_python.LanguageTool(language)

    # Check the text
    matches = tool.check(input_text)

    # Generate corrected text
    corrected_text = language_tool_python.utils.correct(input_text, matches)

    return corrected_text


if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

print(f"### device={device} ###")


if 'pipe' in locals():
    print('Empting gpu cache...')
    del pipe  # Replace 'model' with the name of your existing model variable
    torch.cuda.empty_cache()  # Clear the cache


# pipe = pipeline("text-generation", model="Open-Orca/Mistral-7B-OpenOrca", torch_dtype=torch.bfloat16, device_map=device)
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map=device)


2023-12-01 08:44:38.703246: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 08:44:38.703331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 08:44:38.703404: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 08:44:38.721075: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 08:44:39.691844: W tensorflow/c

### device=cuda:0 ###


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
prompt = """
Du bist {bot_name}, eine KI für lockere Gespräche. Deine Aufgabe: Stelle eine Folgefrage an {user_name}, basierend auf ihren Informationen. {user_name} ist älter.

Beispiel Eingabe:
[
  \"Bob: Ich habe meinen Besuch in Stuttgart geliebt.\"
]

Beispiel Ausgabe:
{bot_name}: Hallo Bob, hier ist {bot_name}! Hast du Zeit zum Plaudern? Was hast du in Stuttgart erlebt?"

Eingabe:
[
  \"{bot_name}: Was hast du denn?\",
  \"Hilde: Ach, meine Beine tun heute weh, und das Wetter macht es nicht besser.\",
  \"{bot_name}: Du hast Beinschmerzen und das Wetter ist dir nicht gut? Wie kann ich dir helfen?\",
  \"Hilde: Keine Ahnung... Es ist, als ob sie mit dem Wetter reagieren. So ein Mist.\"
]

Ausgabe:
{bot_name}:
""".format(user_name='Hilde', bot_name='Adele')


# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": prompt }    
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=128,
                    do_sample=True, temperature=0.01, top_k=100, top_p=0.99
               )
print(outputs[0]["generated_text"])

# Example usage
response = extract_bot_reply(outputs, bot_name='Adele')
print(response)

<|system|>

Du bist Adele, eine KI für lockere Gespräche. Deine Aufgabe: Stelle eine Folgefrage an Hilde, basierend auf ihren Informationen. Hilde ist älter.

Beispiel Eingabe:
[
  "Bob: Ich habe meinen Besuch in Stuttgart geliebt."
]

Beispiel Ausgabe:
Adele: Hallo Bob, hier ist Adele! Hast du Zeit zum Plaudern? Was hast du in Stuttgart erlebt?"

Eingabe:
[
  "Adele: Was hast du denn?",
  "Hilde: Ach, meine Beine tun heute weh, und das Wetter macht es nicht besser.",
  "Adele: Du hast Beinschmerzen und das Wetter ist dir nicht gut? Wie kann ich dir helfen?",
  "Hilde: Keine Ahnung... Es ist, als ob sie mit dem Wetter reagieren. So ein Mist."
]

Ausgabe:
Adele:
</s>
<|assistant|>
Du bist Hilde, eine ältere Person mit Beinschmerzen, die von dem Wetter beeinträchtigt wird. Adele, eine KI für lockere Gespräche, fragt, wie sie dir helfen kann. Was kann ich für dich tun, um dir bei den Beinschmerzen und dem Wetter zu helfen? Wie kann ich dir eine Lösung für diese Probleme vorschlagen?
None


In [5]:
# # few_shot = """
# # Beispiele für Ausgangsszenarien:

# # # Offen für Gespräche:
# # {bot_name}: Guten Tag, Jeniffer! Hier ist {bot_name}. Können wir über jemanden sprechen, den du magst?
# # Jeniffer: Ja, Robert ist sehr lieb zu mir.
# # {bot_name}: Das klingt wunderbar, Jeniffer. Wie lange kennt ihr euch schon?

# # # Ablehnungen:
# # {bot_name}: Hallo Jeniffer, hier ist {bot_name}. Hast du Zeit für ein Gespräch?
# # Jeniffer: Nein, heute nicht...
# # {bot_name}: Verstehe, Jeniffer. Ich bin da, wenn du reden willst. Einen schönen Tag noch!

# # # Verwirrt:
# # {bot_name}: Hallo Jeniffer, hier ist {bot_name}! Können wir über dein Wohlbefinden sprechen?
# # Jeniffer: Wer bist du?
# # {bot_name}: Ich bin {bot_name}, deine Gesundheitsassistentin. Ich bin hier, um dir zuzuhören und dir zu helfen.
# # """.format(user_name='Hilde', bot_name='Adele')


# # few_shot = ""

# few_shot = """
# # Beispiele für Ausgangsszenarien:
# {bot_name}: Hallo Jeniffer! Hier ist {bot_name}. Hast du gut geschlafen?
# Jeniffer: Nein, ich war aufgeregt wegen des Besuchs von Robert.
# {bot_name}: Schade, aber das ist normal! Wer ist Robert?
# """.format(user_name='Hilde', bot_name='Adele')


# # Verwende IMMER den Namen des Benutzers '{user_name}'.

# prompt = """
# Du bist eine KI namens '{bot_name}', deine Aufgabe ist es, nett zu sein und immer Du statt Sie zu verwenden! Gib nur kurze Antworten, weniger als 20 Wörter.

# Du interagierst mit dem Benutzer '{user_name}', einer älteren Person.
# {few_shot}

# """.format(user_name='Hilde', bot_name='Adele', few_shot=few_shot)

In [45]:
prompt = """
Du bist ein freundlicher Assistent, der mit älteren Personen Small Talk führt. Deine Aufgabe besteht darin, auf Basis des gegebenen Eingabe Dialogs eine passende und interessante Folgefrage zu stellen, um das Gespräch weiterzuführen.
""".format(user_name='Hilde', bot_name='Adele')

In [46]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display

def extract_bot_reply(outputs, bot_name='Adele'):
    # Extracts raw response from full generated text
    
    raw_text = outputs[0]["generated_text"]

    patterns = ('<|im_start|>assistant\n', '<|assistant|>\n')
    
    for p in patterns:
        if p in raw_text:
            break
    
    raw_response = outputs[0]["generated_text"].split(p)[-1]

    # Define the pattern to extract the bot's reply
    pattern = rf'{bot_name}: ([^\(\n]*)'

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, raw_response)

    # Return the first match stripped of leading/trailing whitespace, or None if no match
    return matches[0].strip() if matches else raw_response

# Initialize the conversation history
messages = [
    {
        "role": "system",
        "content": prompt
    },
    {
        "role": "assistant",
        "content": "Adele: Guten Abend, Hilde! Hier spricht Adele. Wie war dein Tag?"
    }
]

for m in messages:
    print(m.get('content'))

while True:

    # Get user input
    user_input = input("Hilde: ")

    display("Hilde: " + user_input)

    # Break the loop if the user says something like 'exit', 'quit', etc.
    if user_input.lower() in ['exit', 'quit', 'end']:
        break

    # Update the messages list with the new user input and bot response
    messages.extend([
        {"role": "user", "content": f"Hilde: {user_input}"},
    ])
    
    # Format the prompt using the chat template
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Generate the bot's response
    outputs = pipe(prompt, max_new_tokens=256, 
                #    do_sample=True, temperature=0.3, top_k=50, top_p=0.95
                   do_sample=True, temperature=0.01, top_k=100, top_p=0.99
                   )
    bot_response = extract_bot_reply(outputs, bot_name='Adele')

    messages.extend([
        {"role": "assistant", "content": f"Adele: {bot_response}"}
    ])
    
    display("Adele: " + bot_response)





Du bist ein freundlicher Assistent, der mit älteren Personen Small Talk führt. Deine Aufgabe besteht darin, auf Basis des gegebenen Eingabe Dialogs eine passende und interessante Folgefrage zu stellen, um das Gespräch weiterzuführen.

Adele: Guten Abend, Hilde! Hier spricht Adele. Wie war dein Tag?


'Hilde: Ach, meine Beine tun heute weh, und das Wetter macht es nicht besser.'

'Adele: Oh, ich habe gehört, dass die Wettervorhersagen für diese Woche nicht besonders freundlich sind. Könnten Sie mir erzählen, was Sie heute besonders gemacht haben, bevor Sie sich mit den Beinen beschäftigt haben? Es würde mir freuen, Sie etwas weiter über Ihren Tag zu erfahren.'

'Hilde: Keine Ahnung... Es ist, als ob sie mit dem Wetter reagieren. So ein Mist.'

'Adele: Ich kann verstehen, dass das Wetter Ihnen heute nicht viel Freude bereiten kann. Könnten Sie mir erzählen, ob Sie heute etwas Besonderes gesehen oder gelernt haben? Es würde mir freuen, Sie etwas weiter über Ihre Erfahrungen zu erfahren.'

'Hilde: Naja, es ist so, als ob meine Beine mehr schmerzen, wenn es draußen kalt oder regnerisch ist. Keine Ahnung, warum das so ist.'

'Adele: Ich habe gehört, dass die Kälte und der Regen die Muskeln und Gelenke starr machen können und dadurch die Schmerzen verstärken können. Könnten Sie mir erzählen, ob Sie jemals etwas gegen diese Schmerzen versucht haben? Es würde mir freuen, Sie etwas weiter über Ihre Erfahrungen zu erfahren.'

'Hilde: exit'

: 

In [8]:
Ach, meine Beine tun heute weh, und das Wetter macht es nicht besser.

Keine Ahnung... Es ist, als ob sie mit dem Wetter reagieren. So ein Mist.

Naja, es ist so, als ob meine Beine mehr schmerzen, wenn es draußen kalt oder regnerisch ist. Keine Ahnung, warum das so ist.

Ja, sicher, es ist tatsächlich kalt und nass draußen. Vielleicht ist das der Grund für meine Schmerzen.

Weil sie mich wirklich stören. Es ist schwer, den zu genießen, wenn man ständig Schmerzen hat, weißt du?

war nicht so schön
hab schlecht geschlafen
gerne, aber was?
das mache aber ich schon

SyntaxError: incomplete input (3939930801.py, line 29)